In [1]:
import numpy as np

In [2]:
num_pulleys = 4

### Representation used for this code development ###

![](string_coordinate_representation_img_1.JPG)

#### `get_li` ####

For getting inital $L_i$, we assume that the bot is at the `top-center`. With this assumption, we only need to consider `B & D` metrics for calculating $L_i$



In [3]:
def get_Li(B, D):
    """
    The inital length of strings based on B and D
    """
    return 0.5*np.sqrt(B**2 + D**2)

### TODO
 - [ ] We need to account for 360 degree rotation here
    - Are you talking about the decrease in radius each time the pulley rotates completely?
 - [x] We need to have a reverse calculation done, i.e. given a $x_i,y_i,z_i$ & $x_f,y_f,z_f$ we need to find out what the inital and final thetas. But more on that later.

#### `get_final_string_length` ####
Considering that we have an inital length available, `delta_thetas` given, this function would provide what would be the final length if the pulleys are moved by `delta_theta`

$$ L_f = L_i + r_i\Delta\theta_i $$

#### `get_theta_from_string_length` ####
If we have final length $L_f$ and inital string length $L_i$ and the radius of the pulleys, then we can find the `delta_thetas` moved by the pulleys

<h4 style="color:red">NOTE</h4>
Remember, $L_i$ and & $L_f$ are array/list, so the output would also be array/list each corresponding to individual pulleys.

In [4]:
def get_final_string_length(Li, r, delta_thetas=np.zeros((1,num_pulleys))):
    """
    The final length of strings which need to be achieved for 
    reaching a specific location (based on thetas).
    """
    return Li + r*delta_thetas

def get_theta_from_string_length(Lf, Li, r):
    """
    Given some string lengths Lf and initial lengths Li, return
    the rotation of the pulleys
    """
    return (Lf - Li)/r

#### `get_coordinate` ####
Assumption:
We start from the top of the board, therefore the $L_i$ is same for all strings

1. We take the initial $L_i$ for all strings
2. We are given the `thetas` that the pulley moved
3. We use initial $L_i$ and `thetas` to get final lengths

Then we use following formulas for getting coordinates of the final place the bot moved to

\begin{align}
y &= \frac{l_1^2 - l_2^2 + D^2}{2D} \\
x &= \frac{l_2^2 - l_3^2 + B^2}{2B}
\end{align}

\begin{aligned}
l_1^2 &= (0 - x)^2 + (0 - y)^2 + (H - z)^2 \\ &= x^2 + y^2 + (H - z)^2
\end{aligned}

\begin{aligned}
(H - z)^2 = l_1^2 - x^2 - y^2\\
\end{aligned}

$(H - z)^2$ = Hz2 (Python variable)


In [5]:
def get_coordinate(thetas, B, D, H, r):
    """
    thetas: numpy array of rotations, shape: (num_arrays, )
    B, D, H, r: scalar constants
    """
    Li = get_Li(B, D)
    l = get_final_string_length(Li, r, thetas)
    x = (l[1]**2 - l[2]**2 + B**2)/(2*B)
    y = (l[0]**2 - l[1]**2 + D**2)/(2*D)
    Hz2 = l[0]**2 - x**2 - y**2 # (H-z)^2, eqn 1
    # Is it taut?
    # print(l[0]**2 - l[3]**2, l[1]**2 - l[2]**2)
    z = H - np.sqrt(Hz2)
    return np.array([x, y, z])

### `get_thetas` ###
<p style="color:red">We assume that we always start from the `top` center of the construct that you see on the top.</p>

With the above assumption, the inital length of strings would be taken from `get_Li(B,D)` function.
If we are given the final co-ordinates (x,y,z), we can fund the $Lf$ using following formulas

\begin{align}
\begin{aligned}
l_1^2 &= x^2 + y^2 + (H - z)^2
\end{aligned} \\
\begin{aligned}
l_2^2 &= x^2 + (D - y)^2 + (H - z)^2
\end{aligned} \\
\begin{aligned}
l_3^2 &= (B - x)^2 + (D - y)^2 + (H - z)^2
\end{aligned} \\
\begin{aligned}
l_4^2 &= (B - x)^2 + y^2 + (H - z)^2
\end{aligned} \\
\end{align}

python variable $Hz2 = (H - Z)^2$

In [6]:
def get_thetas(coords, B, D, H, r):
    """
    coords: (x, y, z) coordinates
    B, D, H, r: scalar constants
    """
    l = np.zeros((4,))
    x, y, z = coords
    Hz2 = (H - z)**2 # common term
    l[0] = np.sqrt(x**2 + y**2 + Hz2)
    l[1] = np.sqrt(x**2 + (D - y)**2 + Hz2)
    l[2] = np.sqrt((B - x)**2 + (D - y)**2 + Hz2)
    l[3] = np.sqrt((B - x)**2 + y**2 + Hz2)
    return get_theta_from_string_length(l, get_Li(B, D), r)

In [7]:
# Box dimensions
B, D, H = 100, 200, 300
# Pulley radius
r = 0.1
# Pulley locations
Pulleys = [(0, 0, H), (0, D, H), (B, D, H), (B, 0, H)]

Taking some random values of $\Theta$, we can get the corresponding coordinates from `get_coordinates()`.
If $\Theta$ is such that the ropes are taut, we will get back the original $\Theta$ from `get_thetas()`.
If they aren't, the values will be different. Both are demonstrated in the following example. In the first case, $\theta_4$ is clearly different from the value we set, but in the second case, all angles are exactly the same.

### Below is just a cross-check function. ###
1. Each pulley rotated `[10,20,30,40]` rotations respectively, (i.e 1st pulley, 2nd pulley etc.)
2. With these rotations we get the final co-ordinates, remember we assume we start from top of the structure area.
3. While using the final_cordinates we use `get_thetas` function, i.e how much theta is supposed to be moved, this should come back to thetas we decided earlier.

In [8]:
def display(rotations, coords, theta):
    print(f'For rotation = {rotations/(2*np.pi)}')
    x, y, z = coords
    print(f'\tx = {x}, y = {y}, z = {z}')
    print('\tΘ = ', theta)
    print()
    
# Pulleys rotated 10 revolutions, 20, 30, and 40, respectively
rotations = np.array([10, 20, 30, 40])*2*np.pi
coords = get_coordinate(rotations, B, D, H, r)
theta = get_thetas(coords, B, D, H, r)/(2*np.pi)
display(rotations, coords, theta)

# Pulleys rotated 10 revolutions, 20, 20, and 10, respectively
rotations = np.array([10, 20, 20, 10])*2*np.pi
coords = get_coordinate(rotations, B, D, H, r)
theta = get_thetas(coords, B, D, H, r)/(2*np.pi)
display(rotations, coords, theta)

For rotation = [10. 20. 30. 40.]
	x = 41.98822482885034, y = 96.19150450244695, z = 245.88322994588324
	Θ =  [10.        20.        30.        20.5045589]

For rotation = [10. 20. 20. 10.]
	x = 50.0, y = 96.19150450244695, z = 253.18508971098808
	Θ =  [10. 20. 20. 10.]



## A plot of the ropes
Here's how the system should look after the pulleys have done $(100, 200, 200, 100)$ revolutions:

![](rope.png)

Play around with the `rotations` variable below to see how the position varies.

### Note
The following blocks need Jupyter Lab with [the `jupyter-matplotlib` extension](https://github.com/matplotlib/jupyter-matplotlib#installation) installed. This allows interactive plots, so we can pan and zoom on the 3D plots and get more interesting viewpoints.

In [9]:
%matplotlib widget 
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D

rotations = np.array([100, 200, 200, 100])*2*np.pi
coords = get_coordinate(rotations, B, D, H, r)
x, y, z = coords

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for p in Pulleys:
    ax.plot((p[0], x), (p[1], y), zs=[p[2], z])
for p in Pulleys:
    ax.plot((p[0], p[0]), (p[1], p[1]), zs=[p[2], 0], color='black')

ax.set_xlim3d(0, B)
ax.set_ylim3d(0, D)
ax.set_zlim3d(0, H)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# manually tweaked to get a nicer POV
ax.view_init(elev=7, azim=-109)
ax.dist = 7.5

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## An animation of movement of the robot

As stated in the Equations notebook, any motion of the robot is the result a sequence of steps of the pulley stepper motors. We can generate a sequence of steps for each pulley where each step is one of:

- $1$, indicating the pulley rotated forward one step
- $0$, indicating the pulley didn't rotate
- $-1$, indicating the pulley rotated backward one step

Then, if the pulley rotates by $d\theta_{step}$ for each step, we can get sequence of $d\Theta$s, and taking some random initial $\Theta$, a sequence of $\Theta$.

Let us plot the $(x, y, z)$ for each $\Theta$.

As example, the following function rotates two pulleys every step, and two pulleys every other step. So the first two pulleys get rotated 2x more than the other two pulleys. The effect is that the robot traces a parabola.

This is how the final frame of the animation should look like:

![](arc.png)

The violet line traces the path of the robot.

In [10]:
def get_random_steps(N_steps):
    """
    Example stepper. Each step can be one of three values:
    rotate forward (1), don't move (0) or rotate back (-1)
    """
    for i in range(N_steps):
        yield(np.array([1, 1, i%2, i%2]))


dtheta_step = 10*(np.pi/180) # say, 10°
r = 0.1 # radius of pulley

np.random.seed(17648) # fixed seed to generate same steps each time

N_steps = 100
Theta = np.random.randint(3*np.pi, 5*np.pi, (4,))
Point = get_coordinate(Theta, B, D, H, r)

Thetas = [Theta]
Points = [Point]
steps = []
for step in get_random_steps(N_steps):
    steps.append(step)
    # since steps are too small to show in the figure,
    # let's scale the steps by a factor of 5
    dtheta = (dtheta_step*step)*10
    Theta = Thetas[-1] + dtheta
    Point = get_coordinate(Theta, B, D, H, r)
    Points.append(Point)
    Thetas.append(Theta)

In [11]:
print(3*np.pi, 5*np.pi)
print(np.random.randint(3*np.pi, 5*np.pi, (4,)))

9.42477796076938 15.707963267948966
[ 9  9 10 11]


In [12]:
import mpl_toolkits.mplot3d.axes3d as p3

def animate_points(Points):
    N_steps = len(Points) - 1
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Initial settings for the plot
    P0 = Points[0]
    lines = [ax.plot((P[0], P0[0]), (P[1], P0[1]), zs=[P[2], P0[2]])[0] for P in Pulleys]
    path = ax.plot((P0[0], P0[0]), (P0[1], P0[1]), zs=[P0[2], P0[2]])[0]
    # Pulley rods
    plot_base = 200
    for p in Pulleys:
        ax.plot((p[0], p[0]), (p[1], p[1]), zs=[p[2], plot_base], color='black')

    # Creating the animation function and object
    def update_lines(num, Points, lines, path):
        Point = Points[num]
        for i in range(num_pulleys):
            # Redraw each rope
            line = lines[i]
            Pulley = Pulleys[i]
            line.set_data_3d((Pulley[0], Point[0]),
                             (Pulley[1], Point[1]),
                             (Pulley[2], Point[2]))
        # Trace the path of the robot
        x, y, z = path.get_data_3d()
        if num == 0:
            path.set_data_3d([Point[0]], [Point[1]], [Point[2]])
        else:
            path.set_data_3d(np.append(x, Point[0]),
                             np.append(y, Point[1]),
                             np.append(z, Point[2]))
        return lines
    line_ani = animation.FuncAnimation(fig, update_lines, N_steps, fargs=(Points, lines, path),
                                       repeat=False, blit=False)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_xlim3d(0, B)
    ax.set_ylim3d(0, D)
    ax.set_zlim3d(plot_base, H)
    # manually tweaked to get a nicer POV
    ax.view_init(elev=8, azim=-108)
    ax.dist = 8
    return line_ani, fig

In [19]:
ani, fig = animate_points(Points)
ani.save('anim.gif', writer='imagemagick', fps=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

A GIF of the above animation:

![](anim.gif)

Ultimately, what we need is a sequence of steps that gets us from point A to point B. This is complicated by many things:

- maintaining stability of the platform
- maintaining tension in the ropes
- accounting for rope elasticity in the above
- varying pulley radii, etc.

Keeping reality aside, and keeping to the simple model we have used so far, a simple way to get the steps would be:

1. Get $\Theta$ for both points, and from there, $\Delta\Theta$.
2. Scale $\Delta\Theta$ by $d\theta_{step}$, to get the total step count for each pulley.
3. Take the maximum step count, call it in $N$. This will be the number of elements in the output sequence.
4. Distribute each step count over $N$ elements evenly.

Let's take an example to better understand what (4) means. Let the total step count obtained in (2) be $(6, 3, 3, 1)$. So, pulley 1 has to rotate 6 steps, pulley 2 rotates 3 steps, etc.

Our output, which will be a sequence of individual steps made by each pulley, will have at least 6 elements. It could look like

```
(1, 1, 1, 1)
(1, 1, 1, 0)
(1, 1, 1, 0)
(1, 0, 0, 0)
(1, 0, 0, 0)
(1, 0, 0, 0)
```
But, we'd like the output to be more evenly distributed:

```
(1, 1, 1, 1)        (1, 1, 0, 1)
(1, 0, 0, 0)        (1, 0, 1, 0)
(1, 1, 1, 0)        (1, 1, 0, 0)
(1, 0, 0, 0)   or   (1, 0, 1, 0)
(1, 1, 1, 0)        (1, 1, 0, 0)
(1, 0, 0, 0)        (1, 0, 1, 0)

```

The following simple function does this.

In [14]:
def get_steps_from_dTheta(total_step_counts):
    """
    Return evenly distributed sequence of 1s and 0s
    that sum up to the step counts needed for dTheta
    """
    N = np.int64(np.ceil(max(total_step_counts)))
    Nt = len(total_step_counts)
    steps = [np.zeros(total_step_counts.size) for i in range(N)]
    for j in range(Nt):
        # np.linspace provides evenly spaced numbers
        # we can use these numbers as indexes where steps are made
        for i in np.linspace(0, N - 1, abs(total_step_counts[j])):
            steps[np.int64(i)][j] = 1 if total_step_counts[j] > 0 else -1
    return steps

In [15]:
coords1 = np.array([50, 100, 300])
coords2 = np.array([20, 60, 270])

Theta1 = get_thetas(coords1, B, D, H, r)
Theta2 = get_thetas(coords2, B, D, H, r)

dtheta_step2 = dtheta_step*10
steps = get_steps_from_dTheta((Theta2 - Theta1)/dtheta_step2)
steps[1:10]

[array([-1.,  1.,  1.,  0.]),
 array([-1.,  0.,  1.,  0.]),
 array([-1.,  1.,  1.,  0.]),
 array([0., 1., 1., 0.]),
 array([-1.,  0.,  1.,  0.]),
 array([-1.,  1.,  1.,  0.]),
 array([-1.,  0.,  1., -1.]),
 array([-1.,  1.,  1.,  0.]),
 array([0., 1., 1., 0.])]

In [16]:
Theta = Theta1
Point = get_coordinate(Theta, B, D, H, r)

Thetas = [Theta]
Points2 = [Point]
for step in steps:
    # since steps are too small to show in the figure,
    # let's scale the steps by a factor of 5
    dtheta = (dtheta_step2*step)
    Theta = Thetas[-1] + dtheta
    Point = get_coordinate(Theta, B, D, H, r)
    Points2.append(Point)
    Thetas.append(Theta)

print(Theta2, Thetas[-1])
print(coords1, Points2[0])
print(coords2, Points2[-1])

[-418.03398875  327.64924073  522.08795794  -74.00333786] [-417.13369123  326.37657012  521.85344635  -73.30382858]
[ 50 100 300] [ 50. 100. 300.]
[ 20  60 270] [ 19.85455314  60.12348411 269.94046493]


/Users/muru/dev/agritech/ml/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in sqrt
  del sys.path[0]


As you can see, the final point is quite close to the target point. Let's plot the animation for this. The function isn't perfect - at some steps, the pulleys have been rotated to invalid places. If you look closely, you cna also see that the movement is quite jerky.

In [17]:
ani2, fig2 = animate_points(Points2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …